TEST

In [1]:
##IMPORTS
from __future__ import division
import rhinoinside
rhinoinside.load()
import System
import Rhino.Geometry as rg
from Rhino.Geometry import Point3d, Plane, Brep, Vector3d
from Rhino.Geometry import Surface as RhinoSurface
from Rhino.Geometry import Polyline
from Rhino.Collections import Point3dList
from Rhino.Geometry import PolylineCurve
from Rhino.Geometry import Transform
from Rhino.FileIO import File3dm

import Rhino.UI as rui
import ladybug.epw as epw

from ladybug_geometry.geometry2d.pointvector import Vector2D
from ladybug_radiance.skymatrix import SkyMatrix
from lbt_recipes.version import check_radiance_date

#IMPORTS FOR INCIDENT RADIATION
from ladybug.viewsphere import view_sphere
from ladybug.graphic import GraphicContainer
from ladybug.legend import LegendParameters
from ladybug.color import Colorset

from ladybug_rhino.config import conversion_to_meters
from ladybug_rhino.togeometry import to_joined_gridded_mesh3d, to_vector2d
from ladybug_rhino.fromgeometry import from_mesh3d, from_point3d, from_vector3d
from ladybug_rhino.intersect import intersect_mesh_rays, join_geometry_to_mesh

import ladybug.analysisperiod as ap 

from datetime import datetime, timedelta
import collections
import array
import math
import time
import numpy as np
import matplotlib.pyplot as plt

# check the installed Radiance date and get the path to the gemdaymtx executable
check_radiance_date()

In [4]:
def de_objectify_output(objectified_data):
    """Extract the data from an object that was output from the objectify_output method.

    Args:
        objectified_data: An object that has been output from the objectify_output
            method for which data will be returned.
    """
    return objectified_data.data

In [2]:
_epw_file_path = "FRA_AC_Agen-La.Garenne.AP.075240_TMYx.epw"

epw_data = epw.EPW(_epw_file_path)
location_ = epw_data.location #######
direct_normal_rad_ = epw_data.direct_normal_radiation #########
diffuse_horizontal_rad_ = epw_data.diffuse_horizontal_radiation #########

In [3]:
###SKY MATRIX
# process and set defaults for all of the global inputs
north_ = None
_location = location_
_direct_rad = direct_normal_rad_
_diffuse_rad = diffuse_horizontal_rad_
_folder_ = "C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV/cielMatrices"
 
if north_ is not None:  # process the north_
    try:
        north_ = math.degrees(
            to_vector2d(north_).angle_clockwise(Vector2D(0, 1)))
    except AttributeError:  # north angle instead of vector
        north_ = float(north_)

# create the sky matrix object
sky_mtx_to_IR = SkyMatrix.from_components(
    _location, _direct_rad, _diffuse_rad, 
    #_hoys_, north_, high_density_, ground_r
    )
if _folder_:
    sky_mtx_to_IR.folder = _folder_

In [7]:
start_time2 = time.time()
###########################CODE TO TEST#############################################

skymat = de_objectify_output(sky_mtx_to_IR)

###########################CODE TO TEST#############################################
end_time2 = time.time()
elapsed_time2 = end_time2 - start_time2
print(f"Le temps d'exécution 1 est de {elapsed_time2:.3f} secondes.")

Le temps d'exécution 1 est de 0.000 secondes.


In [ ]:
start_timeap = time.time()
###########################CODE TO TEST#############################################



###########################CODE TO TEST#############################################
end_timeap = time.time()
elapsed_timeap = end_timeap - start_timeap
print(f"Le temps d'exécution 2 est de {elapsed_timeap:.3f} secondes.")